In [ ]:
!pip install pinecone-client

- 파인콘 콘솔 접속(https://app.pinecone.io/)
- API키 메뉴 선택
- API키 복사

In [ ]:
#파인콘 클라이언트 initialize
from pinecone import Pinecone
pc = Pinecone(api_key = '<YOUR_API_KEY>')


In [ ]:

#from pinecone import Pinecone, ServerlessSpec
#pc.create_index(
#    name="quickstart",
#    dimension=8,
#    metric="euclidean",
#    spec=ServerlessSpec(
#        cloud='aws',
#        region='us-west-2'
#    )
#)

In [ ]:
from pinecone import Pinecone, PodSpec
index_name = 'quickstart'
dimension = 8
metric = 'euclidean'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)


index = pc.Index(index_name)

index_stats = index.describe_index_stats()
print(index_stats)

In [ ]:

index.upsert(
  vectors=[
    {"id": "vec1", "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]},
    {"id": "vec2", "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]},
    {"id": "vec3", "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
    {"id": "vec4", "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]}
  ],
  namespace="ns1"
)

index.upsert(
  vectors=[
    {"id": "vec5", "values": [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]},
    {"id": "vec6", "values": [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]},
    {"id": "vec7", "values": [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]},
    {"id": "vec8", "values": [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]}
  ],
  namespace="ns2"
)

In [ ]:
index.describe_index_stats()

In [ ]:
index.query(
  namespace="ns1",
  vector=[0.3,0.3,0.3,0.3,0.4,0.3,0.3,0.3],
  top_k=2,
  include_values=True
)


In [ ]:
index.query(
  namespace="ns2",
  vector=[0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7],
  top_k=3,
  include_values=True
)

# 하이브리드 서치용 메타데이터 Upsert해보기

In [ ]:

index_name = 'quickstart'
dimension = 8
metric = 'euclidean'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

index = pc.Index(index_name)

index_stats = index.describe_index_stats()
print(index_stats)

In [ ]:
index.upsert(
  vectors=[
    {
      "id": "A",
      "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
      "metadata": {"genre":"comedy", "year": 2020}
    },
    {
      "id": "B",
      "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
      "metadata": {"genre":"documentary", "year":2019}
    },
    {
      "id": "C",
      "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
      "metadata": {"genre":"comedy","year":2019}
    },
    {
      "id": "D",
      "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4],
      "metadata": {"genre":"drama"}
    }
  ]
)

- $eq - Equal to (number, string, boolean)

- $ne - Not equal to (number, string, boolean)

- $gt - Greater than (number)

- $gte - Greater than or equal to (number)

- $lt - Less than (number)

- $lte - Less than or equal to (number)

- $in - In array (string or number)

- $nin - Not in array (string or number)

In [ ]:
index.describe_index_stats()

In [ ]:
#쿼리연습
index.query(
    vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    filter={"genre": {"$eq": "documentary"},
            "year": 2019
    },
    top_k=1,
    include_metadata=True
)

In [ ]:
#쿼리연습
index.query(
    vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    filter={"genre": {"$in": ["comedy", "documentary", "drama"]}
        },
    top_k=2,
    include_metadata=True
)


In [ ]:
#쿼리연습
index.query(
    vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    filter={"genre": {"$eq":"documentary"},
            "year" : {"$gte": 2020}

        },
    top_k=2,
    include_metadata=True
)


In [ ]:
#or filter
index.query(
    vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    filter={"$or":[
        {"genre": {"$eq":"documentary"}},
        {"year": {"$gte": 2020}}
    ]

    },
    top_k=2,
    include_metadata=True
)


# Hybrid Search용 Sparse Vector Upsert하기

In [ ]:
index_name = 'quickstart'
dimension = 8
metric = 'dotproduct' # Sparse Vector Operation 지원
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

index = pc.Index(index_name)

index_stats = index.describe_index_stats()
print(index_stats)

In [ ]:
# 파인콘 레코드에 id, 덴스벡터, 메타데이터, 스파스벡터 모두 생성해보기
records=[
    {'id': 'vec1',
     'values': [0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2],
     'metadata': {'genre': 'drama'},
     # index를 제외한 나머지 인덱스는 0
     'sparse_values': {
        'indices': [10, 45, 16],
        'values' : [0.5, 0.5, 0.2]
     }
    },
    {'id': 'vec2',
     'values': [0.2, 0.3, 0.4, 0.2, 0.3, 0.4, 0.9, 0.2],
     'metadata': {'genre': 'action'},
     'sparse_values': {
        'indices' : [15, 40, 11],
        'values' : [0.4, 0.5, 0.2]
     }
    }
]

In [ ]:
# 파인콘 레코드 업서트하기:
index.upsert(records)

In [ ]:
# 시맨틱서치 해보기
index.query(
    top_k=2,
    vector=[0.1, 0.2, 0.3, 0.1, 0.1, 0.2, 0.3, 0.1],
)

In [ ]:
# 하이브리드 서치 해보기 (스파스벡터 추가)
index.query(
    top_k=2,
    vector=[0.1, 0.2, 0.3, 0.1, 0.1, 0.2, 0.3, 0.1],
    sparse_vector={
        'indices' :[10,45,16],
        'values' : [0.5,0.5,0.2]
    }
)

In [ ]:
# Fusion Algorithm으로 하이브리드 결과 스코어링 방식 커스터마이즈
# alpha값으로 덴스<->스파스 가중치 조정
def hybrid_score_norm(dense, sparse, alpha: float):
    hs = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    return [v * alpha for v in dense], hs

hs_dense, hs_sparse = hybrid_score_norm([0.1, 0.2, 0.3, 0.1, 0.1, 0.2, 0.3, 0.1], {
        'indices': [10, 45, 16],
        'values':  [0.5, 0.5, 0.2]
    }, alpha=0.8)
print(hs_dense)
print(hs_sparse)

In [ ]:
# 조정된 가중치 값으로 다시 서치해보기
index.query(top_k=2,vector=hs_dense,sparse_vector=hs_sparse)

# 대용량 데이터 Upsert하기
- 한번의 upsert에 수천,수만개의 벡터를 다 업서트 시키는 대신, 배치화를 통해 업서트 파이프라인 가져가는 방법

In [ ]:

import random
import itertools
def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

vector_dim = 8
vector_count = 500

example_data_generator = map(lambda i: (f'id-{i}', [random.random() for _ in range(vector_dim)]), range(vector_count))

for ids_vectors_chunk in chunks(example_data_generator, batch_size=100):
    index.upsert(vectors=ids_vectors_chunk)

In [ ]:
index.describe_index_stats()

# Fetch / Update / Delete
- 인덱스 된 레코드들에 대한 일련의 CRUD Operation 해보기


In [ ]:
# upsert 시 동봉된 id 값을 이용하여 조회
index.fetch(["vec1","vec2"])

In [ ]:
# 덴스벡터 업데이트

index.update("vec1", values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])

In [ ]:
index.fetch(["vec1"])

In [ ]:
# 메타데이터 업데이트
index.update("vec1", set_metadata={"genre": "Hero", "year":2018})

In [ ]:
index.fetch(["vec1"])

In [ ]:
# sparse vector update
index.update(
	id="vec1",
	sparse_values={"indices": [3,9], "values":[0.5,0.5]}
)

In [ ]:
index.fetch(['vec1'])

In [ ]:
# id값으로 특정 레코드 삭제
index.delete(['vec1'])

In [ ]:
index.fetch(['vec1'])